In [30]:
import os
import sys
from pathlib import Path
import importlib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [43]:
import src.domlm as model 
import src.dataset as dataset
from transformers import AutoTokenizer, AutoModel, DataCollatorForLanguageModeling
importlib.reload(model)
importlib.reload(dataset)

<module 'src.dataset' from '/home/azureuser/dev/DOM-LM/src/dataset.py'>

In [33]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
roberta = AutoModel.from_pretrained("roberta-base")
roberta_config = roberta.config

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
roberta_config_dict = roberta_config.to_dict()
roberta_config_dict["_name_or_path"] = "domlm"
roberta_config_dict["architectures"] = ["DOMLMForMaskedLM"]
domlm_config = model.DOMLMConfig.from_dict(roberta_config_dict)
# domlm_config.save_pretrained("../domlm-config/")
domlm = model.DOMLMForMaskedLM(domlm_config)

Model config DOMLMConfig {
  "_name_or_path": "domlm",
  "architectures": [
    "DOMLMForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "depth_pad_id": 127,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_depth_embeddings": 128,
  "max_node_embeddings": 2048,
  "max_position_embeddings": 514,
  "max_sibling_embeddings": 128,
  "max_tag_embeddings": 128,
  "model_type": "domlm",
  "node_pad_id": 2047,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sibling_pad_id": 127,
  "tag_pad_id": 127,
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



In [45]:
from collections import OrderedDict

state_dict = OrderedDict((f"domlm.{k}",v) for k,v in roberta.state_dict().items())
domlm.load_state_dict(state_dict,strict=False)

_IncompatibleKeys(missing_keys=['domlm.embeddings.tree_embeddings.node_embeddings.weight', 'domlm.embeddings.tree_embeddings.parent_embeddings.weight', 'domlm.embeddings.tree_embeddings.sibling_embeddings.weight', 'domlm.embeddings.tree_embeddings.depth_embeddings.weight', 'domlm.embeddings.tree_embeddings.tag_embeddings.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'], unexpected_keys=['domlm.pooler.dense.weight', 'domlm.pooler.dense.bias'])

In [46]:
train_ds = dataset.SWDEDataset("/home/azureuser/dev/data/swde_preprocessed")
test_ds = dataset.SWDEDataset("/home/azureuser/dev/data/swde_preprocessed",split="test")

In [47]:
# TODO: change datacollator for masking whole nodes
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [48]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=5,
    warmup_ratio=0.1,
    learning_rate=1e-4,
)

trainer = Trainer(
    model=domlm,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
)

trainer.train()